# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
import torch as th
import syft as sy
from torch import nn, optim

hook = sy.TorchHook(th)

In [3]:
# create a couple workers

bob = sy.VirtualWorker(hook, id = 'bob')
alice = sy.VirtualWorker(hook, id = 'alice')
secure_worker = sy.VirtualWorker(hook, id = 'secure_worker')

In [4]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

<VirtualWorker id:secure_worker #tensors:0>

In [29]:
# A Toy Dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad = True)
target = th.tensor([[0], [0], [1], [1.]], requires_grad = True)

In [30]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [31]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [32]:
# Intialize a Toy Model
model = nn.Linear(2, 1)

In [50]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params = bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params = alices_model.parameters(), lr=0.1)

for i in range(10):
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    bobs_loss

    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss
    
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [51]:
with th.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [53]:

for round_iter in range(10):
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params = bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params = alices_model.parameters(), lr=0.1)

    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    with th.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob: " + str(bobs_loss) + " Alice: " + str(alices_loss))

Bob: tensor(0.0034) Alice: tensor(4.8821e-05)
Bob: tensor(0.0017) Alice: tensor(1.0550e-05)
Bob: tensor(0.0010) Alice: tensor(4.6678e-05)
Bob: tensor(0.0006) Alice: tensor(5.9277e-05)
Bob: tensor(0.0004) Alice: tensor(5.5744e-05)
Bob: tensor(0.0003) Alice: tensor(4.6607e-05)
Bob: tensor(0.0002) Alice: tensor(3.6941e-05)
Bob: tensor(0.0002) Alice: tensor(2.8533e-05)
Bob: tensor(0.0001) Alice: tensor(2.1758e-05)
Bob: tensor(9.9184e-05) Alice: tensor(1.6483e-05)
